In [1]:
import requests
import os
from bs4 import BeautifulSoup

In [2]:
client_id=os.environ.get('client_id')
client_secret=environ.get('client_secret')

url='https://api.spotify.com/v1/search?'
url_params='q=muse'+'&type=artist'

access_token=os.environ.get('access_token')
headers= {
    'Authorization': 'Bearer '+access_token
}
response=requests.get(url+url_params,headers=headers)

In [75]:
data=response.json()

In [85]:
data['artists']['items'][0]['uri'][-22:]

'12Chz98pHFMPJEknJQMWvI'

In [5]:
from spotifyAPICaller import find_artist

In [4]:
def find_artist_id(name,token):
    url='https://api.spotify.com/v1/search?'
    url_params=f'q={name}'+'&type=artist'

    access_token=token
    headers= {
        'Authorization': 'Bearer '+access_token
    }
    response=requests.get(url+url_params,headers=headers)
    return response.json()['artists']['items'][0]['uri'][-22:]

In [12]:
find_artist('Panic! At the disco',access_token)

'20JZFwl6HVl6yg8a4H3ZqK'